# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 16 2023 9:17AM,259428,10,SOTAH0000939,"Nextsource Biotechnology, LLC",Released
1,Mar 16 2023 9:17AM,259428,10,SONSB0002138,"Nextsource Biotechnology, LLC",Released
2,Mar 16 2023 9:14AM,259427,20,ATR-I000109,"HVL, LLC dba Atrium Innovations",Released
3,Mar 16 2023 9:11AM,259426,20,9235189-BO,"Exact-Rx, Inc.",Released
4,Mar 16 2023 9:11AM,259426,20,9235184-BO,"Exact-Rx, Inc.",Released
5,Mar 16 2023 9:11AM,259426,20,9239439-BO,"Exact-Rx, Inc.",Released
6,Mar 16 2023 9:11AM,259426,20,9240051-BO,"Exact-Rx, Inc.",Released
7,Mar 16 2023 9:11AM,259426,20,9240053-BO,"Exact-Rx, Inc.",Released
8,Mar 16 2023 9:11AM,259426,20,9240058-BO,"Exact-Rx, Inc.",Released
9,Mar 16 2023 9:11AM,259426,20,9261353-BO,"Exact-Rx, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
12,259424,Released,1
13,259425,Released,8
14,259426,Released,26
15,259427,Released,1
16,259428,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
259424,NaN,1.0
259425,NaN,8.0
259426,NaN,26.0
259427,NaN,1.0
259428,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
259379,0.0,1.0
259404,1.0,0.0
259413,29.0,17.0
259414,0.0,54.0
259415,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
259379,0,1
259404,1,0
259413,29,17
259414,0,54
259415,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,259379,0,1
1,259404,1,0
2,259413,29,17
3,259414,0,54
4,259415,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,259379,,1
1,259404,1,
2,259413,29,17
3,259414,,54
4,259415,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 16 2023 9:17AM,259428,10,"Nextsource Biotechnology, LLC"
2,Mar 16 2023 9:14AM,259427,20,"HVL, LLC dba Atrium Innovations"
3,Mar 16 2023 9:11AM,259426,20,"Exact-Rx, Inc."
29,Mar 16 2023 9:08AM,259425,10,"Digital Brands, LLC"
37,Mar 16 2023 8:58AM,259424,10,"Citieffe, Inc."
38,Mar 16 2023 8:55AM,259423,10,"Methapharm, Inc."
45,Mar 16 2023 8:53AM,259421,10,"Methapharm, Inc."
46,Mar 16 2023 8:51AM,259420,10,Hush Hush
66,Mar 16 2023 8:37AM,259419,16,Sartorius Bioprocess Solutions
67,Mar 16 2023 8:30AM,259404,10,Yusen – 3D Matrix


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Mar 16 2023 9:17AM,259428,10,"Nextsource Biotechnology, LLC",,2
1,Mar 16 2023 9:14AM,259427,20,"HVL, LLC dba Atrium Innovations",,1
2,Mar 16 2023 9:11AM,259426,20,"Exact-Rx, Inc.",,26
3,Mar 16 2023 9:08AM,259425,10,"Digital Brands, LLC",,8
4,Mar 16 2023 8:58AM,259424,10,"Citieffe, Inc.",,1
5,Mar 16 2023 8:55AM,259423,10,"Methapharm, Inc.",,7
6,Mar 16 2023 8:53AM,259421,10,"Methapharm, Inc.",,1
7,Mar 16 2023 8:51AM,259420,10,Hush Hush,,20
8,Mar 16 2023 8:37AM,259419,16,Sartorius Bioprocess Solutions,1,
9,Mar 16 2023 8:30AM,259404,10,Yusen – 3D Matrix,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 16 2023 9:17AM,259428,10,"Nextsource Biotechnology, LLC",2,
1,Mar 16 2023 9:14AM,259427,20,"HVL, LLC dba Atrium Innovations",1,
2,Mar 16 2023 9:11AM,259426,20,"Exact-Rx, Inc.",26,
3,Mar 16 2023 9:08AM,259425,10,"Digital Brands, LLC",8,
4,Mar 16 2023 8:58AM,259424,10,"Citieffe, Inc.",1,
5,Mar 16 2023 8:55AM,259423,10,"Methapharm, Inc.",7,
6,Mar 16 2023 8:53AM,259421,10,"Methapharm, Inc.",1,
7,Mar 16 2023 8:51AM,259420,10,Hush Hush,20,
8,Mar 16 2023 8:37AM,259419,16,Sartorius Bioprocess Solutions,,1
9,Mar 16 2023 8:30AM,259404,10,Yusen – 3D Matrix,,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 16 2023 9:17AM,259428,10,"Nextsource Biotechnology, LLC",2,
1,Mar 16 2023 9:14AM,259427,20,"HVL, LLC dba Atrium Innovations",1,
2,Mar 16 2023 9:11AM,259426,20,"Exact-Rx, Inc.",26,
3,Mar 16 2023 9:08AM,259425,10,"Digital Brands, LLC",8,
4,Mar 16 2023 8:58AM,259424,10,"Citieffe, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Mar 16 2023 9:17AM,259428,10,"Nextsource Biotechnology, LLC",2.0,NaN
1,Mar 16 2023 9:14AM,259427,20,"HVL, LLC dba Atrium Innovations",1.0,NaN
2,Mar 16 2023 9:11AM,259426,20,"Exact-Rx, Inc.",26.0,NaN
3,Mar 16 2023 9:08AM,259425,10,"Digital Brands, LLC",8.0,NaN
4,Mar 16 2023 8:58AM,259424,10,"Citieffe, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 16 2023 9:17AM,259428,10,"Nextsource Biotechnology, LLC",2.0,0.0
1,Mar 16 2023 9:14AM,259427,20,"HVL, LLC dba Atrium Innovations",1.0,0.0
2,Mar 16 2023 9:11AM,259426,20,"Exact-Rx, Inc.",26.0,0.0
3,Mar 16 2023 9:08AM,259425,10,"Digital Brands, LLC",8.0,0.0
4,Mar 16 2023 8:58AM,259424,10,"Citieffe, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1815945,39.0,1.0
16,259419,0.0,1.0
18,518794,2.0,0.0
19,778243,77.0,29.0
20,518853,27.0,0.0
22,259418,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1815945,39.0,1.0
1,16,259419,0.0,1.0
2,18,518794,2.0,0.0
3,19,778243,77.0,29.0
4,20,518853,27.0,0.0
5,22,259418,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,39.0,1.0
1,16,0.0,1.0
2,18,2.0,0.0
3,19,77.0,29.0
4,20,27.0,0.0
5,22,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,39.0
1,16,Released,0.0
2,18,Released,2.0
3,19,Released,77.0
4,20,Released,27.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,18,19,20,22
Status,,,,,,
Executing,1.0,1.0,0.0,29.0,0.0,1.0
Released,39.0,0.0,2.0,77.0,27.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,18,19,20,22
0,Executing,1.0,1.0,0.0,29.0,0.0,1.0
1,Released,39.0,0.0,2.0,77.0,27.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,18,19,20,22
0,Executing,1.0,1.0,0.0,29.0,0.0,1.0
1,Released,39.0,0.0,2.0,77.0,27.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()